# Dump UVW array to Single File

File formats are:

* A) Create 2D slices into single ZIP file (ext = ".ZIP")
* B) Simple JSON file with single big list (ext = ".JSON")
* C) CSV file with i,j,k plus (u,v,w) pixel (ext = ".CSV")  ...but not efficient although semantical rich
* D) Text file with first line = shape (eg, (128,128,128,3)) followed by lines of RGB triplets (ext = ".TXT")


In [8]:
import numpy as np
import xarray as xr
import glob
import imageio
import os
from zipfile import ZipFile  # needed for format A

### not needed so far
#import json, codecs
#import pandas as pd
#from PIL import Image

## Start with Small UVW Example IF Needed

In [4]:
#ERROR!!!! COMMENT if simple example is needed. Then skip next sections setting uvw from NetCDF dataset

size = 2
u = np.random.randint(0,256,(size,size,size))  # minmic pixel RGB colors 
v = np.random.randint(0,256,(size,size,size)) 
w = np.random.randint(0,256,(size,size,size)) 

# combine with np.stack making the new dimension the last
uvw = np.stack((u, v, w), axis=-1).astype(np.uint8)

print('WRITTEN: uvw type=', type(uvw), type(uvw[0,0,0,0]), 'shape=', uvw.shape, ' # cells=', np.prod(uvw.shape), 'sum=', uvw.sum())

WRITTEN: uvw type= <class 'numpy.ndarray'> <class 'numpy.uint8'> shape= (2, 2, 2, 3)  # cells= 24 sum= 3035


## Read the NetCDF Dataset as Xarray object

In [21]:
# define config constants
file_name = "tornado3d"
data_path = "../data/"
extension = "nc"

# open dataset with Xarray & dump
ds = xr.open_dataset(data_path + file_name + '.' + extension)
ds

<xarray.Dataset>
Dimensions:  (xdim: 128, ydim: 128, zdim: 128)
Coordinates:
  * xdim     (xdim) float32 -10.0 -9.84252 -9.6850395 ... 9.6850395 9.84252 10.0
  * ydim     (ydim) float32 -10.0 -9.84252 -9.6850395 ... 9.6850395 9.84252 10.0
  * zdim     (zdim) float32 -10.0 -9.84252 -9.6850395 ... 9.6850395 9.84252 10.0
Data variables:
    u        (zdim, ydim, xdim) float32 ...
    v        (zdim, ydim, xdim) float32 ...
    w        (zdim, ydim, xdim) float32 ...
Attributes:
    creation_date:  16-Apr-2019 17:36:10
    grid:           regular

## Convert to u,v,w ndarrays, combine into single uvw array, Normalize/Rescale to RGB pixels

In [22]:
u = np.array(ds.data_vars['u'])
v = np.array(ds.data_vars['v'])
w = np.array(ds.data_vars['w'])

# combine with np.stack making the new dimension the last
uvw_raw = np.stack((u, v, w), axis=-1)

## Find min-max of u-v-w velocities
uvw_min = np.amin(uvw_raw)
uvw_max = np.amax(uvw_raw)
print('min=', uvw_min, 'max=', uvw_max)

## Normalize and scale for RGB color using x_norm = 256*(X-uvw_min)/(uvw_max-uvw_min)
uvw = (255 * (uvw_raw / (uvw_max-uvw_min) - (uvw_min/(uvw_max-uvw_min)))).astype(np.uint8)

print('WRITTEN: uvw type=', type(uvw), type(uvw[0,0,0,0]), 'shape=', uvw.shape, ' # cells=', np.prod(uvw.shape), 'sum=', uvw.sum())

min= -6.2111263 max= 6.2132716
WRITTEN: uvw type= <class 'numpy.ndarray'> <class 'numpy.uint8'> shape= (128, 128, 128, 3)  # cells= 6291456 sum= 804248128


## A) Create ZIP of 2D slices, along with animated GIF of those slices

Need Work: ZIP file has extra nested directories before accessing JPG slices

In [ ]:
import os
import tempfile

path = 'log/'        #os.getcwd()
fileName = r'\JPEG_Slice'

# create a temporary directory
with tempfile.TemporaryDirectory() as tempPath:
    # print('The created temporary directory is %s' % directory)

    # slice on the first index. change to second/third index to slice on others 2D
    (no_slices,_,_,_) = uvw.shape 

    # write JPG files to temp directory
    for i in range(no_slices):
        # create a temp directory under path
        rgbFile = tempPath + fileName + '{:04d}.jpg'.format(i)
        imageio.imwrite(rgbFile, uvw[i,:,:,:])   # note imwrite interprets last dim as 3 bytes for RGB24
        
    fileList = sorted(glob.glob(tempPath + fileName + '*.jpg'))

    # create the ZIP archive 
    with ZipFile(path + fileName + 's.ZIP', 'w') as zipObj:
        # Iterate over all the files in directory
        for file in fileList:
            # Add file to zip
            zipObj.write(file)

    # create animated GIF of JPG images
    with imageio.get_writer(path + fileName + 's.GIF', mode='I') as writer:
        for file in fileList:
            image = imageio.imread(file)
            writer.append_data(image)

        image = imageio.imread(file)
        writer.append_data(image)


## B. Create simple JSON file with single big list

In [88]:
ext = ".JSON2"
file_path = "log/uvw_test1" + ext

# Write file as single big list
uvw_str = json.dumps(uvw.tolist())
json.dump(uvw_str, codecs.open(file_path, 'w', encoding='utf-8'), separators=(',',':'), sort_keys=True, indent=4)

print('SENT: uvw type=', type(uvw), type(uvw[0,0,0,0]), 'shape=', uvw.shape, ' # cells=', np.prod(uvw.shape), 'sum=', uvw.sum())

SENT: uvw type= <class 'numpy.ndarray'> <class 'numpy.uint8'> shape= (128, 128, 128, 3)  # cells= 6291456 sum= 804248128


In [56]:
######################## test file
import ast

# Read file back and Reconstruct UVW array
file_text = codecs.open(file_path, 'r', encoding='utf-8').read()
uvw_list = ast.literal_eval(json.loads(file_text))
#print(type(uvw_list))
uvw2 = np.array(uvw_list)

print('LOADED: uvw type=', type(uvw), type(uvw[0,0,0,0]), 'shape=', uvw.shape, ' # cells=', np.prod(uvw.shape), 'sum=', uvw.sum())

LOADED: uvw type= <class 'numpy.ndarray'> <class 'numpy.uint8'> shape= (4, 4, 4, 3)  # cells= 192 sum= 24833


## C. Create CSV file with i,j,k plus (u,v,w) pixel

NOTE: very inefficient => IGNORE

In [63]:
import csv
ext = ".CSV"
file_path = "log/uvw_test1" + ext

with open(file_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(['x', 'y', 'z', 'u', 'v', 'w'])
    
    for i in range(uvw.shape[0]):
        for j in range(uvw.shape[1]):
            for k in range(uvw.shape[2]):
                line = [i, j, k, uvw[i,j,k,0], uvw[i,j,k,1], uvw[i,j,k,2]]
                print(line)
                writer.writerow(line)
    f.close()

[0, 0, 0, 154, 46, 203]
[0, 0, 1, 105, 38, 217]
[0, 1, 0, 200, 236, 77]
[0, 1, 1, 204, 240, 158]
[1, 0, 0, 251, 195, 171]
[1, 0, 1, 88, 221, 0]
[1, 1, 0, 203, 218, 239]
[1, 1, 1, 79, 179, 47]


## D. Create text file with first line as shape, following by lines for each 3-int pixel

In [86]:
ext = ".TXT"
file_path = "log/uvw_test1" + ext
sep = " "
eol = '\n'

with open(file_path, "w") as f:
    line = str(uvw.shape[0]) +' '+ str(uvw.shape[1]) +' '+ str(uvw.shape[2]) + '\n'  # first row is UVW shape dimensions
    f.write(line)
    
    for i in range(uvw.shape[0]):
        for j in range(uvw.shape[1]):
            for k in range(uvw.shape[2]):
                line = str(uvw[i,j,k,0]) +sep+ str(uvw[i,j,k,1]) +sep+ str(uvw[i,j,k,2]) + eol
                f.write(line)
    f.close()
    
print('SENT: uvw type=', type(uvw), type(uvw[0,0,0,0]), 'shape=', uvw.shape, ' # cells=', np.prod(uvw.shape), 'sum=', uvw.sum())

SENT: uvw type= <class 'numpy.ndarray'> <class 'numpy.uint8'> shape= (128, 128, 128, 3)  # cells= 6291456 sum= 804248128


In [87]:
################# test file
ext = ".TXT"
file_path = "log/uvw_test1" + ext

with open(file_path, "r") as f:
    line = f.readline()        # first row is UVW shape dimensions
    (shape0, shape1, shape2) = tuple(map(int, line.split()))
    uvw2 = np.ones((shape0, shape1, shape2, 3)).astype(np.uint8)
    
    for i in range(shape0):
        for j in range(shape1):
            for k in range(shape2):
                line = f.readline()
                uvw2[i, j, k, :] = tuple(map(int, line.split()))
    f.close()
    
print('SENT: uvw type=', type(uvw2), type(uvw2[0,0,0,0]), 'shape=', uvw2.shape, ' # cells=', np.prod(uvw2.shape), 'sum=', uvw2.sum())

SENT: uvw type= <class 'numpy.ndarray'> <class 'numpy.uint8'> shape= (128, 128, 128, 3)  # cells= 6291456 sum= 804248128


## FOR LATER... Pieces from Eric, packing hex bytes into RGB 

In [ ]:
from eric May 6

uvw_norm = (255 * (uvw / (uvw_max-uvw_min) - (uvw_min/(uvw_max-uvw_min)))).astype(np.int8)

flattened = uvw_norm.flatten().tobytes()
newFile = open("../data/flattened.txt", "wb")
newFile.write(flattened)
newFile.close()

In [56]:
# Method B - packing bytes in hex
import struct

rgb = uvw_norm[1,1,1,:]
print(rgb)
rgb_hex = bytes.hex(struct.pack('BBB',*rgb))
int(rgb_hex, 16)

[125  87 127]


8214399

## FOR LATER... Define function to list all files in directory with a specific extension

In [24]:


def list_files(directory, extension):
    return (f for f in listdir(directory) if f.endswith('.' + extension))

# List each nc file with its creation data and variables
for f in list_files(data_path, netCDF_extension):
    print(f)
    nc = Dataset(data_path + f, "r")
    nc_dict = nc.__dict__
    cdate = nc_dict['creation_date']
    vars = 
    print(f"{f} created {cdate} with variables {vars}")


tornado3d.nc
tornado3d.nc created 16-Apr-2019 17:36:10
